# App

## Instructions

- Run all till installtions in before itself.
- Add templates after running till installation go to `./drive/MyDrive/certificate/templates` add the template of the certificate (as .pptx).
- Update the `Template` variable in code under generate certificate and asign the key for the value of the file name (not the path).
- Now add the credential for the Email, Add the email to EMAIL and app password to PASS.

## Connecting

Connecting google drive with google colab.

In [ ]:
!mkdir drive
from google.colab import drive
drive.mount('/content/drive')

mkdir: cannot create directory ‘drive’: File exists
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Installation

In [ ]:
!pip install requests python-pptx==0.6.22 Spire.Presentation==8.9.0 Spire==0.4.2 secure-smtplib pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 71.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 18.8 MB/s eta 0:00:00
  Created wheel for Spire: filename=spire-0.4.2-py3-none-any.whl size=1575493 sha256=faab708debac08ba48ea44513d42b722ddd639b3b715dc915657494acf5282c4
  Stored in directory: /root/.cache/pip/wheels/64/19/05/c80428de5fce514dd790337b4f404a798906db9641bca4dc79
Successfully built Spire


In [ ]:
import os
if not os.path.exists("./files"):
    os.mkdir("./files")
ppt = "./drive/MyDrive/certificate/files/ppt"
if not os.path.exists(ppt):
    os.mkdir(ppt)
pdf = "./drive/MyDrive/certificate/files/pdf"
if not os.path.exists(pdf):
    os.mkdir(pdf)

if not os.path.exists("./drive/MyDrive/certificate/templates"):
    os.mkdir("./drive/MyDrive/certificate/templates")

## Code

### Generate Certificate

In [ ]:
import smtplib, requests, os
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pptx import Presentation as XPresentation
from spire.presentation import *
from spire.presentation.common import *

Template = {
    1: "test"
}
def genCertificate(template: int, value):
    presentation = XPresentation(f'./drive/MyDrive/certificate/templates/{Template[template]}.pptx')
    for slide in presentation.slides:
        for shape in slide.shapes:
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        run.text = run.text.replace("{name}", value['name'])
                        run.text = run.text.replace("{id}", value['id'])
    presentation.save(f'{ppt}/{value["id"]}.pptx')
    result = ppt2pdf(value)
    try:
        os.remove(f'{ppt}/{value["id"]}.pptx')
    except Exception:
        pass
    try:
        os.remove(f'{pdf}/{value["id"]}.pdf')
    except Exception:
        pass
    return result

def ppt2pdf(value):
    presentation = Presentation()
    presentation.LoadFromFile(f"{ppt}/{value['id']}.pptx")
    slide = presentation.Slides[0]
    slide.SaveToFile(f"{pdf}/{value['id']}.pdf", FileFormat.PDF)
    presentation.Dispose()
    rs = requests.post("https://certify.izaries.workers.dev/certificate",
        data=value,
        files={'file': open(f"{pdf}/{value['id']}.pdf", 'rb')}
    ).json()
    if rs["success"]:
        return { "success": True, "dl": f"https://certify.izaries.workers.dev/download/{rs['id']}" }
    else:
        return { "success": False, "message": rs["message"] }


EMAIL = ""
PASS = ""


def sendEmail(name: str, email: str, result):
    try:
        html_content = f"""
        <html>
          <head></head>
          <body>
          {result}
          </body>
        </html>
        """
        message = MIMEMultipart()
        message["From"] = EMAIL
        message["To"] = email
        message["Subject"] = "SOSC: Certificate of Participation"
        message.attach(MIMEText(html_content, "html"))
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(EMAIL, PASS)
        server.sendmail(EMAIL, email, message.as_string())
        server.quit()
        return {"success": True}
    except Exception:
        return {"success": False}

excel_file = '/content/data.xlsx'
df = pd.read_excel(excel_file)

for i in df['SL No'].unique().tolist():
    try:
        result = genCertificate(1, {"name": df['Name'][i-1], "id": df['ID'][i-1]})
        if result["success"] and sendEmail(df['Name'][i-1], df['Email'][i-1], result)["success"]:
            df['Link'][i-1] = result["dl"]
            df['Sent'][i-1] = True
            print(f"Successfully sent to {df['Name'][i-1]} ")
        else:
            print(f"Failed Name: {df['Name'][i-1]} with {result['message']}")
    except Exception as e:
        print(f"Failed Name: {df['Name'][i-1]} with {e}")

df.to_excel(excel_file, index=False)

df = pd.read_excel(excel_file)
for i in df['SL No'].unique().tolist():
    if not bool(df['Sent'][i-1]):
        print(f"Was unable to send {df['Email'][i-1]} of {df['Name'][i-1]}")


